# Описание проекта

## Цели проекта

Повышение эффективности клиентоориентированной стратегии банка через сегментацию клиентов на основе их поведения (RFM-анализ) для:

- Увеличения удержания клиентов (Retention Rate).
- Роста прибыльности за счет таргетированных предложений.
- Снижения оттока (Churn Rate) в группах риска.

## Задачи

1. **Сбор и подготовка данных**:

   - Получить исторические данные о транзакциях, продуктах, взаимодействиях клиентов за последние 12–24 месяца.
   - Очистить данные от аномалий (например, тестовые операции, технические ошибки).

2. **Расчет RFM-метрик**:

   - **Recency (R)**: Как давно клиент совершал операции (дни с последней транзакции).
   - **Frequency (F)**: Частота взаимодействий (число операций за период).
   - **Monetary (M)**: Суммарный доход банка от клиента (комиссии, проценты по кредитам/депозитам).

3. **Сегментация клиентов**:

   - Разделить клиентов на группы по комбинациям R, F, M (например, «Лояльные», «Уходящие», «Спящие»).
   - Выявить сегменты с наибольшим потенциалом для кросс-продаж.

4. **Анализ сегментов**:

   - Сравнить средние значения метрик (ARPU, LTV, Churn Rate) между сегментами.
   - Изучить распределение клиентов по банковским продуктам (кредиты, вклады, карты).

5. **Проверка гипотез**:

   - Определить, какие гипотезы подтвердились, и сформулировать рекомендации.

6. **Разработка рекомендаций**:
   - Создать персонализированные маркетинговые кампании для каждого сегмента.
   - Предложить условия для удержания клиентов из групп риска.

## Гипотезы

1. **Гипотеза 1**: Клиенты с высоким **Monetary** (M) чаще используют несколько продуктов банка (кредит + депозит + страхование).

   - _Метод проверки_: Сравнение среднего числа продуктов в сегментах.

2. **Гипотеза 2**: Клиенты с низким **Recency** (R ≤ 30 дней) имеют в 2 раза выше Retention Rate, чем клиенты с R > 90 дней.

   - _Метод проверки_: Анализ Retention Rate за последний квартал.

3. **Гипотеза 3**: Сегмент «Спящие клиенты» (R > 180 дней, F = 1) можно реактивировать email-рассылкой с персональными условиями.

   - _Метод проверки_: A/B-тест с предложением бонусов за возврат.

4. **Гипотеза 4**: Клиенты с высоким **Frequency** (F ≥ 5) более лояльны: их NPS на 20% выше, чем у клиентов с F ≤ 2.

   - _Метод проверки_: Сопоставление данных RFM с опросами NPS.

5. **Гипотеза 5**: Клиенты, пользующиеся мобильным банком, имеют более высокий **Monetary** за счет удобства совершения операций.
   - _Метод проверки_: Сравнение M у пользователей мобильного банка и остальных.


# Import


In [3]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta


In [5]:
# Настройки
np.random.seed(42)
n_transactions = 5000
n_customers = 1000
n_products = 50
start_date = datetime(2023, 3, 1)
end_date = datetime(2024, 3, 1)

In [6]:
# Генерация данных
data = {
    "transaction_date": [
        start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days),
                              hours=np.random.randint(0, 24),
                              minutes=np.random.randint(0, 60))
        for _ in range(n_transactions)
    ],
    "customer_id": [f"user_{np.random.randint(1, n_customers+1)}" for _ in range(n_transactions)],
    "amount": np.round(np.random.exponential(scale=500, size=n_transactions) + 100),  # Экспоненциальное распределение
    "product_id": [f"prod_{np.random.randint(1, n_products+1)}" for _ in range(n_transactions)],
    "payment_method": np.random.choice(
        ["карта", "Apple Pay", "Google Pay", "криптовалюта", "другое"],
        size=n_transactions,
        p=[0.6, 0.15, 0.1, 0.1, 0.05]
    ),
    "is_anomaly": 0
}

df = pd.DataFrame(data)

In [7]:
# Добавление аномалий (5% транзакций)
anomaly_mask = np.random.rand(n_transactions) < 0.05
df.loc[anomaly_mask, "amount"] = np.round(np.random.uniform(10000, 50000, size=anomaly_mask.sum()), 2)
df.loc[anomaly_mask, "is_anomaly"] = 1

In [8]:
# Сохранение
df.to_csv("transactions_rfm_extended.csv", index=False)

In [9]:
df.head()

,transaction_date,customer_id,amount,product_id,payment_method,is_anomaly
0,2023-06-11 19:28:00,user_165,166.0,prod_40,Google Pay,0
1,2023-11-26 10:07:00,user_878,516.0,prod_39,Apple Pay,0
2,2023-09-05 20:38:00,user_294,1621.0,prod_44,карта,0
3,2023-06-30 18:22:00,user_911,888.0,prod_49,карта,0
4,2024-01-25 10:23:00,user_519,532.0,prod_44,карта,0
